# CPSC 532S Assigment 5(b): Graph Neural Networks 

The goal of this part of the assignment is to get you familiar with implementing graph neural network in pytorch. Recently, there has been an explosion of research in the field of graph neural network. In this assignment we will start with the vanilla variant of graph neural network and develop other variants such as gated GNN and Graph Attention Networks. 

### Setup 

You will need to install pytroch_geometric for this part of the assigment.

"*PyTorch Geometric (PyG) is a geometric deep learning extension library for PyTorch.
It consists of various methods for deep learning on graphs and other irregular structures, also known as geometric deep learning, from a variety of published papers. In addition, it consists of an easy-to-use mini-batch loader for many small and single giant graphs, multi gpu-support, a large number of common benchmark datasets (based on simple interfaces to create your own), and helpful transforms, both for learning on arbitrary graphs as well as on 3D meshes or point clouds*"

To install pytorch geometric follow the instructions below:

1. Determine the pytorch version and CUA version pytroch was installed with

In [ ]:
import os
import torch 
print('Pytorch Version:',torch.__version__); print('CUDA version:',torch.version.cuda)

Pytorch Version: 1.7.1
CUDA version: 10.1


Now we will set some environment variable that will indicate which version of the pytorch gemetric binaries(and dependencies) we will have to install. 

You will need a pytroch with version >= 1.4.0. You will have to set the `$TORCH` variable to one of(`1.4.0`, `1.5.0`, `1.6.0`, `1.7.0`, `1.8.0`). For a pytroch version of the form `1.x.y` set the `$TORCH` variable to `1.x.0` in the cell below. For example, I have pytroch `1.7.1` so i set the variable to `1.7.0`. 

Similarly of the cuda versions, set the `$CUDA` variable to on of (`cpu`, `cu92`, `cu101`, `cu102`, `cu110`, `cu111`). The number at the end indicate the cuda version. If you have a cuda version `10.1` then use `cu101`.

**Make sure you set this appropiately or else the libraires wont work.**

In [ ]:
os.environ['TORCH'] = '1.7.0'
os.environ['CUDA'] = 'cu101'

Install the relevant packages using the command below.

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric

The main reason we are using Pytorch Geometric of this assignemt is beacuse it has a large collection of datasets that can be readily used. Additionally, they have implementation of Dataloader which can be tricky in case of graph as batching graphs with varying number of nodes is not as easy as stacking in case of images.

Please go through https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html to understand how Pytroch Geometric handles graph data

## Dataset

In this assignment we will use Cora dataset.

"*The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.*"

The nodes in the graph are publications and edges correspond to citations: if publication A cites publication B, then the graph has an edge from A to B. The task here is node classification where we want to classify each paper into one of seven classes :
 - `Case_Based` 
 - `Genetic_Algorithms`
 - `Neural_Networks`
 - `Probabilistic_Methods`
 - `Reinforcement_Learning`
 - `Rule_Learning`
 - `Theory`
 
 The dataset is readily available in pytorch geometric and we will use that to load the data.

In [ ]:
from torch_geometric.datasets import Planetoid
path = 'datasets/'
dataset = Planetoid(root=path, name='Cora')
print('Number of graphs in the dataset   :',len(dataset))
print('Number of nodes the entire graph  :', dataset[0].x.shape[0])
print('Numver of nodes in the train set  :', dataset[0].train_mask.sum().item())
print('Numver of nodes in the val   set  :', dataset[0].val_mask.sum().item())
print('Numver of nodes in the test  set  :', dataset[0].test_mask.sum().item())

Number of graphs in the dataset   : 1
Number of nodes the entire graph  : 2708
Numver of nodes in the train set  : 140
Numver of nodes in the val   set  : 500
Numver of nodes in the test  set  : 1000


**NOTE:** The dataset one contains one giant graph. The dataset is split into train, val and test in terms of the nodes i.e. we will use the the labels of some of the nodes (specified by train mask) to train our network and we will test by classifying the nodes in the test set. This is slightly different from what you are generally in case of image or text data. Also bear in mind that not all graph datasets will in the form of one giant graph. For example when working with molecular dataset each graph will correspond to a chemical compound and your dataset will have a collection of disjoint graphs.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch import Tensor
from torch_scatter import gather_csr, scatter
from torch_geometric.utils import add_self_loops, degree, softmax
import torch_geometric.transforms as T

## Graph Neural Network

We will first set up a base class that will act as a parent for subsequent convolution layers that you will implement. Please go through the base class to understand what each method in the class is used for.

In [ ]:
class MessagePassing(nn.Module):
    r"""
    Base Class for implementing Message Passing Layers.
    A message passing layer has three main components:
        1. Message    : Specifies what infromation should 
                        be passed out from a node.
        2. Aggreagate : Specifies how infromation will be 
                        propogated to neighbours.
        3. Combine    : Specifies how the incoming message 
                        will be used to update the current state.
    Note: We will only focus on graph with features only on nodes
          in this assignment. It should give you a good idea
          of how to implement message passing layer and how to
          extend it to graphs with edge features.
    
    """
    def __init__(self, node_dim: int = -2, aggr: str = "add"):
            super().__init__()
            self.node_dim = node_dim
            self.aggr = aggr
    
    def message(self, node_states: Tensor, edge_index) -> Tensor:
        r"""
        From the nodes states contruct the message the
        goes out to all the neighbours.
        This step usually involves passing the nodes
        states though some linear layer. Once transformed
        you will have to transfrom the states to messages
        over the edges.
        Args:
        -----
            nodes_states: A tensor containing the states of
                          nodes in a graph
            edge_index: A 2xE tensor specifying the connectivity 
                        structure of a graph.
                        Ex: For the graph below
                                  0
                                /   \
                               1     2
                            edge_index = [
                                            [0, 0],
                                            [1, 2]
                            ]
                            The first colum specifies the edge
                            (0,1) and the second column specifies
                            the edge (0,2).
                        The matrix is specified this was rather 
                        than a nxn adjacency matrix with 0 and 1
                        entries for memory and computational 
                        efficiency. Adjacency matrices are
                        often quite sparse and represting them
                        as dense matrices can lead to waste of
                        memory and compute time.
        Returns:
        --------
            A tensor that transforms the nodes states to the
            required message
        """
        #For the base class we will just return the nodes 
        #states as is.
        return node_states[edge_index[0]]
    
    def aggregate(self, messages: Tensor = None, edge_index: Tensor = None, dim_size = None) -> Tensor:
        r"""
        For each node aggregate the messages from its neighbours
        Args:
        -----
            messages  : The transformed nodes staes
            edge_index: A 2xE tensor specifying the connectivity 
                        structure of a graph.
            dim_size  : Number of nodes in the graph
        Returns:
        -------
            A tensor corresponding to aggregates messages
        """
        raise NotImplemented
    
    def combine(self, node_states: Tensor, agg_message:Tensor) -> Tensor:
        r"""
        A function to combine the incomming message and the current
        state of the nodes.
        Args:
        ----
            nodes_states: The current states of the nodes
            agg_message : Messages aggregated from its neighbours
        Returns:
        --------
            Updates nodes states
        """
        raise NotImplemented

In [ ]:
class GraphConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels):
        super(GraphConv, self).__init__()
        
        self.kernel = nn.Linear(in_channels, out_channels)
    
    def messages(self, node_states, edge_index, edge_weights):
        '''
        Function to convert node states to message over the edges
        Args:
        ----
            node_states  : Node features
            edge_index   : 2xE tensor specifying graph structure
            edge_weights : Weights to normalize the messages.
        '''
        
        messages = None
        # Your code goes here (note that this can be implemented in as little as 3 lines of code)


        #####################
        return messages
    
    def aggregate(self, messages, edge_index, dim_size):
        '''
        Function to convert node states to message over the edges
        Args:
        ----
            node_states  : Node features
            edge_index   : 2xE tensor specifying graph structure
            edge_weights : Weights to normalize the messages.
        '''
        
        # Your code goes here (note that this can be implemented in 1-2 lines of code)
        #Hint: You will have to use the scatter function from torch_scatter


        #####################
        return out
        
    def combine(self, old_states, new_states):
        return new_states
    
    def compute_norm(self, edge_index, dim_size):
        row, col = edge_index
        deg = degree(col, dim_size, dtype=edge_index.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        return norm
    
    def forward(self, node_states, edge_index):
        #Add self loops into the graph
        edge_index, _ = add_self_loops(edge_index, num_nodes=node_states.size(0))
        #Compute number of nodes in the graph
        dim_size = node_states.size(self.node_dim)
        #Compute the normalization constants
        norm = self.compute_norm(edge_index, dim_size)
        #Create the messages from the node states
        messages = self.messages(node_states.clone(), edge_index, norm)
        #Aggregate the node states
        new_states = self.aggregate(messages, edge_index, dim_size)
        #Combine the new states with the old one
        return self.combine(node_states, new_states)

Now let implement the Graph Convolution Network (GCN) as proposed in [Semi-Supervised Classification with Graph Convolutional 
Networks](https://arxiv.org/abs/1609.02907).

The convolution operation in a GCN is given by
$$Z = \hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2} W X$$
where $$\hat{A} = A + I$$ is the adjaceny matrix augmented with self loops and D is a diagonal degree matrix with diagonal entries(D_ii) equal to number of edge outgoing from node i. W is the kernel matrix that transform the node states X.

We have defined the `GraphConv` layer that inherits `MessagePassing` below. You have to implement `message` and `aggregate` methods.

We will define a graph convolution network using the network you implemented above.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GraphConv(dataset.num_node_features, 16)
        self.conv2 = GraphConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [ ]:
best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))

Epoch: 001, Train: 0.7071, Val: 0.4880, Test: 0.4800
Epoch: 002, Train: 0.7571, Val: 0.4880, Test: 0.4800
Epoch: 003, Train: 0.7929, Val: 0.4880, Test: 0.4800
Epoch: 004, Train: 0.8143, Val: 0.4880, Test: 0.4800
Epoch: 005, Train: 0.8429, Val: 0.4880, Test: 0.4800
Epoch: 006, Train: 0.8500, Val: 0.4880, Test: 0.4800
Epoch: 007, Train: 0.8643, Val: 0.5000, Test: 0.5200
Epoch: 008, Train: 0.9071, Val: 0.5380, Test: 0.5570
Epoch: 009, Train: 0.9571, Val: 0.6020, Test: 0.6160
Epoch: 010, Train: 0.9643, Val: 0.6540, Test: 0.6530
Epoch: 011, Train: 0.9857, Val: 0.6820, Test: 0.6840
Epoch: 012, Train: 0.9857, Val: 0.7040, Test: 0.7220
Epoch: 013, Train: 0.9857, Val: 0.7360, Test: 0.7430
Epoch: 014, Train: 0.9857, Val: 0.7400, Test: 0.7580
Epoch: 015, Train: 0.9857, Val: 0.7540, Test: 0.7650
Epoch: 016, Train: 0.9857, Val: 0.7580, Test: 0.7790
Epoch: 017, Train: 0.9857, Val: 0.7680, Test: 0.7850
Epoch: 018, Train: 0.9857, Val: 0.7680, Test: 0.7850
Epoch: 019, Train: 0.9857, Val: 0.7700, Test: 

Epoch: 177, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 178, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 179, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 180, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 181, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 182, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 183, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 184, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 185, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 186, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 187, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 188, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 189, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 190, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 191, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 192, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 193, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 194, Train: 1.0000, Val: 0.7760, Test: 0.7890
Epoch: 195, Train: 1.0000, Val: 0.7760, Test: 

**The expected test accuracy is > 75%.**

## Graph Attention Networks 

Next we will implement the [Graph Attention Network](https://arxiv.org/abs/1710.10903) introduced in this [paper](https://arxiv.org/abs/1710.10903). We will start by defining additional initialization functions.

In [ ]:
import math
def glorot(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)
def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)

You will have to fill in the `messages` and `aggregate` method for Graph Attention Network below. Note: We will define a graph attention Layer with a single head.

In [ ]:
class GraphAttention(MessagePassing):
    
    def __init__(self, in_channels, 
                 out_channels,
                 negative_slope=0.2,
                 dropout=0, bias=True
                ):
        
        super(GraphAttention, self).__init__(node_dim=0)
        
        self.out_channels = out_channels
        self.negative_slope = negative_slope
        self.dropout = dropout
        
        self.kernel = nn.Linear(in_channels, out_channels, bias=False)
        
        self.att_l = nn.Parameter(torch.Tensor(1, out_channels))
        self.att_r = nn.Parameter(torch.Tensor(1, out_channels))
        
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        
        self.reset_parameters()
    
    def reset_parameters(self):
        glorot(self.kernel.weight)
        glorot(self.att_l)
        glorot(self.att_r)
        zeros(self.bias)
    
    def messages(self, node_states, edge_index):
        '''
        Function to convert node states to message over the edges
        Args:
        ----
            node_states  : Node features
            edge_index   : 2xE tensor specifying graph structure
        '''
        C = self.out_channels
        num_nodes = node_states.shape[0]
        
        node_states = self.kernel(node_states).view(-1, C)
        
        alpha_l = ... # Your code
        alpha_r = ... # Your code
        
        # Compute messages 
        
        # Compute edge attention
        alpha = .... # Your code for unnormalized attention
        index = .... # Destination index of the node for messages

        alpha = softmax(alpha, index, None, num_nodes)
        self._alpha = alpha

        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return messages * alpha.unsqueeze(-1)
    
    def aggregate(self, messages, edge_index, dim_size):
        '''
        Function to convert node states to message over the edges
        Args:
        ----
            node_states  : Node features
            edge_index   : 2xE tensor specifying graph structure
            edge_weights : Weights to normalize the messages.
        '''   
        # Your code goes here (note that this can be implemented in 1-2 lines of code)
        # Hint: You will have to use the scatter function from torch_scatter
        # This is identical to the standard GNN above (you can simply copy it over)


        #####################
        return out

    def combine(self, old_states, new_states):
        return new_states
    
    def forward(self, node_states, edge_index):       
        #Add self loops into the graph
        edge_index, _ = add_self_loops(edge_index, num_nodes=node_states.size(0))
        #Compute number of nodes in the graph
        dim_size = node_states.size(self.node_dim)
        
        #Create the messages from the node states
        messages = self.messages(node_states.clone(), edge_index)
        #Aggregate the node states
        new_states = self.aggregate(messages, edge_index, dim_size)
        new_states = new_states.view(-1, self.out_channels)

        if self.bias is not None:
            new_states += self.bias
            
        #Combine the new states with the old one
        return self.combine(node_states, new_states)

In [ ]:
dataset = Planetoid('datasets/', 'Cora', transform=T.NormalizeFeatures())
data = dataset[0]

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GraphAttention(dataset.num_features, 64, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GraphAttention(8 * 8, dataset.num_classes, dropout=0.6)

    def forward(self, data):
        x = F.dropout(data.x, p=0.6, training=self.training)
        # import ipdb; ipdb.set_trace()
        x = F.elu(self.conv1(x, data.edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [ ]:
for epoch in range(1, 201):
    train()
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, *test()))

Epoch: 001, Train: 0.9857, Val: 0.7780, Test: 0.8030
Epoch: 002, Train: 0.9857, Val: 0.7780, Test: 0.8030
Epoch: 003, Train: 0.9857, Val: 0.7760, Test: 0.8030
Epoch: 004, Train: 0.9857, Val: 0.7760, Test: 0.8020
Epoch: 005, Train: 0.9857, Val: 0.7720, Test: 0.7980
Epoch: 006, Train: 0.9857, Val: 0.7720, Test: 0.7970
Epoch: 007, Train: 0.9857, Val: 0.7740, Test: 0.7990
Epoch: 008, Train: 0.9857, Val: 0.7800, Test: 0.8030
Epoch: 009, Train: 0.9857, Val: 0.7840, Test: 0.8030
Epoch: 010, Train: 0.9929, Val: 0.7840, Test: 0.8060
Epoch: 011, Train: 0.9929, Val: 0.7880, Test: 0.8060
Epoch: 012, Train: 0.9929, Val: 0.7860, Test: 0.8080
Epoch: 013, Train: 0.9929, Val: 0.7940, Test: 0.8140
Epoch: 014, Train: 0.9929, Val: 0.7960, Test: 0.8190
Epoch: 015, Train: 0.9929, Val: 0.8040, Test: 0.8210
Epoch: 016, Train: 0.9929, Val: 0.8060, Test: 0.8220
Epoch: 017, Train: 0.9929, Val: 0.8080, Test: 0.8240
Epoch: 018, Train: 0.9929, Val: 0.8080, Test: 0.8190
Epoch: 019, Train: 0.9929, Val: 0.8060, Test: 

Epoch: 183, Train: 1.0000, Val: 0.7960, Test: 0.8150
Epoch: 184, Train: 0.9929, Val: 0.7920, Test: 0.8150
Epoch: 185, Train: 0.9929, Val: 0.7900, Test: 0.8130
Epoch: 186, Train: 0.9929, Val: 0.7900, Test: 0.8100
Epoch: 187, Train: 0.9929, Val: 0.7900, Test: 0.8090
Epoch: 188, Train: 0.9929, Val: 0.7880, Test: 0.8080
Epoch: 189, Train: 0.9929, Val: 0.7900, Test: 0.8020
Epoch: 190, Train: 0.9929, Val: 0.7860, Test: 0.7990
Epoch: 191, Train: 0.9929, Val: 0.7860, Test: 0.7940
Epoch: 192, Train: 0.9929, Val: 0.7780, Test: 0.7920
Epoch: 193, Train: 0.9929, Val: 0.7800, Test: 0.7910
Epoch: 194, Train: 0.9929, Val: 0.7860, Test: 0.8060
Epoch: 195, Train: 0.9929, Val: 0.7920, Test: 0.8110
Epoch: 196, Train: 0.9929, Val: 0.7940, Test: 0.8130
Epoch: 197, Train: 0.9929, Val: 0.7960, Test: 0.8160
Epoch: 198, Train: 0.9929, Val: 0.7960, Test: 0.8200
Epoch: 199, Train: 1.0000, Val: 0.7980, Test: 0.8240
Epoch: 200, Train: 1.0000, Val: 0.8000, Test: 0.8240
